In [1]:
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
from spotipy.oauth2 import SpotifyOAuth

cid = '5d9494efb79d4abaabc13e9778431f49'
secret = '4d23d535c7fe4e9cadd241ddf58df15e'
client_credentials_manager = SpotifyClientCredentials(
    client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


In [2]:
# timeit library to measure the time needed to run this code
import timeit
start = timeit.default_timer()

# create empty lists where the results are going to be stored
artist_name = []
track_name = []
popularity = []
track_id = []

for i in range(0, 1000, 50):
    track_results = sp.search(q='year:2018', type='track', limit=50, offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])


stop = timeit.default_timer()
print('Time to run this code (in seconds):', stop - start)


Time to run this code (in seconds): 8.651795899990248


In [3]:
track_name


['Whiskey Glasses',
 'I Was Never There',
 'Drip Too Hard (Lil Baby & Gunna)',
 'Yes Indeed',
 'SLOW DANCING IN THE DARK',
 'Call Out My Name',
 'All Girls Are The Same',
 "Chasin' You",
 'Lucid Dreams',
 'Pure Cocaine',
 'ball w/o you',
 'lovely (with Khalid)',
 'a lot',
 'Softcore',
 'Beautiful Crazy',
 'Hope',
 'Telephones',
 'rockstar (feat. 21 Savage)',
 'You Get Me So High',
 'Washing Machine Heart',
 'Lean Wit Me',
 'Shoota (feat. Lil Uzi Vert)',
 'Violent Crimes',
 'All The Stars (with SZA)',
 'SAD!',
 "God's Plan",
 'Self Care',
 'Armed And Dangerous',
 'Going Bad (feat. Drake)',
 'we fell in love in october',
 'Taking A Walk',
 '10 Freaky Girls (with 21 Savage)',
 'Little Dark Age',
 'Moonlight',
 'Space Cadet (feat. Gunna)',
 'Stir Fry',
 'Sunflower - Spider-Man: Into the Spider-Verse',
 'Show Me How',
 'She Got the Best of Me',
 'SICKO MODE',
 'Ghost Town',
 '1400 / 999 Freestyle',
 'Hex',
 'After The Storm (feat. Tyler, The Creator & Bootsy Collins)',
 'Life Goes On',
 'Be

In [4]:
print('number of elements in the track_id list:', len(track_id))


number of elements in the track_id list: 1000


In [5]:
import pandas as pd

df_tracks = pd.DataFrame({'artist_name': artist_name, 'track_name': track_name,
                         'track_id': track_id, 'popularity': popularity})
print(df_tracks.shape)
df_tracks.head()


(1000, 4)


,artist_name,track_name,track_id,popularity
0,Morgan Wallen,Whiskey Glasses,6foY66mWZN0pSRjZ408c00,80
1,The Weeknd,I Was Never There,1cKHdTo9u0ZymJdPGSh6nq,89
2,Lil Baby,Drip Too Hard (Lil Baby & Gunna),78QR3Wp35dqAhFEc2qAGjE,84
3,Lil Baby,Yes Indeed,6vN77lE9LK6HP2DewaN6HZ,82
4,Joji,SLOW DANCING IN THE DARK,0rKtyWc8bvkriBthvHKY8d,84


In [6]:
df_tracks.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   artist_name  1000 non-null   object
 1   track_name   1000 non-null   object
 2   track_id     1000 non-null   object
 3   popularity   1000 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 31.4+ KB


In [7]:
# group the entries by artist_name and track_name and check for duplicates

grouped = df_tracks.groupby(
    ['artist_name', 'track_name'], as_index=True).size()
grouped[grouped > 1].count()


11

In [8]:
df_tracks.drop_duplicates(subset=['artist_name', 'track_name'], inplace=True)


In [9]:
# doing the same grouping as before to verify the solution
grouped_after_dropping = df_tracks.groupby(
    ['artist_name', 'track_name'], as_index=True).size()
grouped_after_dropping[grouped_after_dropping > 1].count()


0

In [10]:
df_tracks[df_tracks.duplicated(
    subset=['artist_name', 'track_name'], keep=False)].count()


artist_name    0
track_name     0
track_id       0
popularity     0
dtype: int64

In [11]:
df_tracks.shape


(989, 4)

In [12]:
# again measuring the time
start = timeit.default_timer()

# empty list, batchsize and the counter for None results
rows = []
batchsize = 100
None_counter = 0

for i in range(0, len(df_tracks['track_id']), batchsize):
    batch = df_tracks['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)

print('Number of tracks where no audio features were available:', None_counter)

stop = timeit.default_timer()
print('Time to run this code (in seconds):', stop - start)


Number of tracks where no audio features were available: 0
Time to run this code (in seconds): 2.5249419000028865


In [13]:
print('number of elements in the track_id list:', len(rows))


number of elements in the track_id list: 989


In [14]:
df_audio_features = pd.DataFrame.from_dict(rows, orient='columns')
print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.head()


Shape of the dataset: (989, 18)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.614,0.680,6,-4.580,1,0.0289,0.3690,0.000002,0.115,0.707,149.959,audio_features,6foY66mWZN0pSRjZ408c00,spotify:track:6foY66mWZN0pSRjZ408c00,https://api.spotify.com/v1/tracks/6foY66mWZN0p...,https://api.spotify.com/v1/audio-analysis/6foY...,234347,4
1,0.323,0.737,10,-4.008,1,0.0338,0.1420,0.000162,0.163,0.155,113.260,audio_features,1cKHdTo9u0ZymJdPGSh6nq,spotify:track:1cKHdTo9u0ZymJdPGSh6nq,https://api.spotify.com/v1/tracks/1cKHdTo9u0Zy...,https://api.spotify.com/v1/audio-analysis/1cKH...,241067,4
2,0.897,0.662,1,-6.903,0,0.2920,0.0852,0.000000,0.534,0.389,112.511,audio_features,78QR3Wp35dqAhFEc2qAGjE,spotify:track:78QR3Wp35dqAhFEc2qAGjE,https://api.spotify.com/v1/tracks/78QR3Wp35dqA...,https://api.spotify.com/v1/audio-analysis/78QR...,145543,4
3,0.963,0.346,5,-9.309,0,0.5300,0.0355,0.000000,0.108,0.562,119.957,audio_features,6vN77lE9LK6HP2DewaN6HZ,spotify:track:6vN77lE9LK6HP2DewaN6HZ,https://api.spotify.com/v1/tracks/6vN77lE9LK6H...,https://api.spotify.com/v1/audio-analysis/6vN7...,142273,4
4,0.515,0.479,3,-7.458,1,0.0261,0.5440,0.005980,0.191,0.284,88.964,audio_features,0rKtyWc8bvkriBthvHKY8d,spotify:track:0rKtyWc8bvkriBthvHKY8d,https://api.spotify.com/v1/tracks/0rKtyWc8bvkr...,https://api.spotify.com/v1/audio-analysis/0rKt...,209274,4


In [15]:
df_audio_features.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 989 entries, 0 to 988
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      989 non-null    float64
 1   energy            989 non-null    float64
 2   key               989 non-null    int64  
 3   loudness          989 non-null    float64
 4   mode              989 non-null    int64  
 5   speechiness       989 non-null    float64
 6   acousticness      989 non-null    float64
 7   instrumentalness  989 non-null    float64
 8   liveness          989 non-null    float64
 9   valence           989 non-null    float64
 10  tempo             989 non-null    float64
 11  type              989 non-null    object 
 12  id                989 non-null    object 
 13  uri               989 non-null    object 
 14  track_href        989 non-null    object 
 15  analysis_url      989 non-null    object 
 16  duration_ms       989 non-null    int64  
 1

In [16]:
columns_to_drop = ['analysis_url', 'track_href', 'type', 'uri']
df_audio_features.drop(columns_to_drop, axis=1, inplace=True)

df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)

df_audio_features.shape


(989, 14)

In [17]:
birdy_uri = 'spotify:artist:2WX2uTcsvV5OnS0inACecP'

results = sp.artist_albums(birdy_uri, album_type='album')
albums = results['items']
while results['next']:
    results = sp.next(results)
    albums.extend(results['items'])

for album in albums:
    print(album['name'])


Young Heart
Beautiful Lies
Beautiful Lies
Beautiful Lies (Deluxe)
Beautiful Lies (Deluxe)
Fire Within
Fire Within
Fire Within (Deluxe)
Fire Within (Deluxe)
Fire Within (Deluxe)
Live in London
Birdy
Birdy
Birdy
Birdy
Birdy (Deluxe Version)


In [18]:
# merge both dataframes
# the 'inner' method will make sure that we only keep track IDs present in both datasets
df = pd.merge(df_tracks, df_audio_features, on='track_id', how='inner')
print("Shape of the dataset:", df_audio_features.shape)
df.head()


Shape of the dataset: (989, 14)


,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Morgan Wallen,Whiskey Glasses,6foY66mWZN0pSRjZ408c00,80,0.614,0.680,6,-4.580,1,0.0289,0.3690,0.000002,0.115,0.707,149.959,234347,4
1,The Weeknd,I Was Never There,1cKHdTo9u0ZymJdPGSh6nq,89,0.323,0.737,10,-4.008,1,0.0338,0.1420,0.000162,0.163,0.155,113.260,241067,4
2,Lil Baby,Drip Too Hard (Lil Baby & Gunna),78QR3Wp35dqAhFEc2qAGjE,84,0.897,0.662,1,-6.903,0,0.2920,0.0852,0.000000,0.534,0.389,112.511,145543,4
3,Lil Baby,Yes Indeed,6vN77lE9LK6HP2DewaN6HZ,82,0.963,0.346,5,-9.309,0,0.5300,0.0355,0.000000,0.108,0.562,119.957,142273,4
4,Joji,SLOW DANCING IN THE DARK,0rKtyWc8bvkriBthvHKY8d,84,0.515,0.479,3,-7.458,1,0.0261,0.5440,0.005980,0.191,0.284,88.964,209274,4


In [19]:
df[df.duplicated(subset=['artist_name', 'track_name'], keep=False)]


,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature


In [20]:
df.to_csv('SpotifyAudioFeatures08082018.csv')


In [29]:
# import libraries
import glob
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# get all csv files into one variable
path = 'SpotifyAudioFeatures08082018'
all_files = glob.glob(os.path.join(path, "*.csv"))

# create lists of columns to be used when reading/merging the csv's
columns = ['artist_name', 'track_id', 'track_name', 'popularity']
merge_columns = ['artist_name', 'track_id', 'track_name']

# create dataframes by reading the csv's in all_files
df_from_each_file = (pd.read_csv(f, usecols=columns) for f in all_files)

# create empty dataframe with the defined column structure
df = pd.DataFrame(columns=columns)

# loop over dataframes and merge into one dataframe
# outer join in order to keep the popularity column from each file
# all_files are here to provide the column suffix (0920,0830 etc)
for df_, files in zip(df_from_each_file, all_files):
    df = df.merge(df_, how='outer', on=merge_columns,
                  suffixes=('', str(files)[-8:-4]))

print('Shape: ', df.shape)
df.head()


Shape:  (0, 4)


,artist_name,track_id,track_name,popularity


In [ ]:
# sum individual popularity scores
df['popularity'] = df[['popularity0920',
                       'popularity0830', 'popularity0807']].sum(axis=1)

# calculate also the mean popularity score
df['popularity_mean'] = df[['popularity0920',
                            'popularity0830', 'popularity0807']].mean(axis=1)

# create new dataframe df_top ordered consisting of the 100 most popular tracks
df_top = df.sort_values('popularity', ascending=False).head(100)

# show the first 50 results
df_top[['artist_name', 'track_name', 'popularity', 'popularity_mean']].head(50)


In [27]:
# show top 20 artists by number of tracks in top 100
df_top[['artist_name', 'track_name']].groupby(
    'artist_name').count().sort_values('track_name', ascending=False).head(20)


NameError: name 'df_top' is not defined

In [ ]:
# create a new transposed dataframe where the track names are the columns and individual popularities the rows
df_top10_pop = df_top[['track_name', 'popularity0807', 'popularity0830',
                       'popularity0920']].set_index('track_name').head(10).T

# set the figure size
plt.figure(figsize=(12, 18))

# create a color palette
palette = plt.get_cmap('Set1')

# multiple line plot of the top10 track popularities
num = 0
for track in df_top10_pop.columns:
    num += 1

    # find the right spot on the plot
    plt.subplot(10, 1, num)

    # plot the individual popularities
    df_top10_pop.loc[['popularity0807', 'popularity0830', 'popularity0920'], track].plot(
        marker='', color=palette(num), linewidth=2.5)

    # same limits for every subplot
    plt.ylim(90, 100)

    # get current position of the ticks
    locs, labels = plt.xticks()

    # add ticks with custom labels
    mylabels = ['', '7th August', '', '', '', '30th August',
                '', '', '', '20th September']  # a bit ugly but it works
    plt.xticks(locs, mylabels)

    # not ticks everywhere
    if num in range(10):
        plt.tick_params(labelbottom=False)

    # add title
    plt.title(track, loc='left', fontsize=10, fontweight=0, color=palette(num))

# add general title
plt.suptitle("Popularity of Top 10 Tracks during Summer 2018",
             fontsize=13, fontweight=0, color='black', style='italic')
